In [1]:
import json
from collections import Counter
import pandas as pd
import joblib


In [3]:
# Langkah 1: Baca Data JSON
with open('student_validations.json') as file:
    data = json.load(file)
    
# Ekstrak data yang relevan
table_data = data[2]['data']

In [4]:
# Langkah 2: Ekstrak Informasi dan Hitung Frekuensi
report_counts = []
error_counts = []

for record in table_data:
    testid = int(record['testid'])
    status = record['status']
    report_lines = record['report'].split('\n')
    
    if status == 'FAILED':
        for line in report_lines:
            if '-> FAILED ->' in line:
                report_counts.append((testid, status, line.strip()))
    elif status == 'ERROR':
        for line in report_lines:
            if line.strip():
                error_counts.append((testid, status, line.strip()))

# Hitung frekuensi kemunculan setiap jenis laporan kesalahan
report_counter = Counter(report_counts)
error_counter = Counter(error_counts)

# Konversi hasil ke dalam DataFrame
report_df = pd.DataFrame(
    [(testid, status, report, count) for (testid, status, report), count in report_counter.items()],
    columns=['testid', 'status', 'report', 'count']
)

error_df = pd.DataFrame(
    [(testid, status, report, count) for (testid, status, report), count in error_counter.items()],
    columns=['testid', 'status', 'report', 'count']
)

# Gabungkan DataFrame untuk FAILED dan ERROR
combined_df = pd.concat([report_df, error_df], ignore_index=True)

In [5]:
# Langkah 3: Simpan Hasil
# Simpan DataFrame ke CSV (atau format lain jika diperlukan)
combined_df.to_csv('combined_report_counts.csv', index=False)

# Simpan model yang dilatih (misal: DecisionTreeClassifier)
# Latih ulang model jika diperlukan
X = combined_df[['testid', 'count']]
y = combined_df['report']

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X, y)

# Simpan model ke file pkl
joblib.dump(model, 'combined_report_model.pkl')

print("Proses selesai, file 'combined_report_counts.csv' dan 'combined_report_model.pkl' telah dibuat.")

Proses selesai, file 'combined_report_counts.csv' dan 'combined_report_model.pkl' telah dibuat.
